In [2]:
import numpy as np
import pandas as pd
import scipy
from scipy import special
from scipy.special import erf

In [3]:
#Black-Scholes model
r=0.02
sigm=0.11
tau=180/365
S=95
K=105

def Phi(x):
    return 0.5*(1+scipy.special.erf(x/np.sqrt(2)))

def N1():
    a=np.log(S/K) + (r+sigm*sigm/2)*tau
    a=a/(sigm*np.sqrt(tau))
    return Phi(a)


def N2():
    a=np.log(S/K) + (r-sigm*sigm/2)*tau
    a=a/(sigm*np.sqrt(tau))
    return Phi(a)

def price():
    return S*N1()-K*N2()*np.exp(-r*tau)

price()

0.45956819386863046

In [7]:
#оцениваем sigm
df=pd.read_csv("USD_EUR_year.csv")
otv=0
for i in range(len(df)-2, -1, -1):
    otv+=(float(df['Цена'].values[i][2:])/float(df['Цена'].values[i+1][2:])-1)**2
np.sqrt(otv)

0.06090956841760476

In [8]:
#Garmam and Kolhagen model for usd/euro
import scipy
from scipy.stats import norm as normal_distrib

def MyPhi(x):
    return normal_distrib.cdf(x,0,1)
def myphi(x):
        return normal_distrib.pdf(x,0,1)
    
class FXPricer():
    def __init__(self,r_d,r_f,sigm,T,S0,K):
        self.r_d=r_d
        self.r_f=r_f
        self.sigm=sigm
        self.T=T
        self.S0=S0
        self.K=K
    def d1(self):
        return (np.log(self.S0/self.K) + ((self.r_d-self.r_f)+self.sigm*self.sigm/2)*self.T)/(self.sigm*np.sqrt(self.T))
    def d2(self):
        return self.d1()-self.sigm*np.sqrt(self.T)
    def get_price(self):
        return self.S0*np.exp(-self.r_f*self.T)*MyPhi(self.d1())-self.K*np.exp(-self.r_d*self.T)*MyPhi(self.d2())
    def get_delta(self):
        return np.exp(-self.r_f*self.T)*MyPhi(self.d1())
    def get_gamma(self):
        return np.exp(-self.r_f*self.T)*myphi(self.d1())/(self.S0*self.sigm*np.sqrt(self.T))
    def get_theta(self):
        return -(self.S0*self.sigm*myphi(self.d1()))/(2*np.sqrt(self.T))
    
    
MyPricer=FXPricer(r_d=1.24/100, r_f=2.29/100, sigm=0.06, T=365/365, S0=0.88,K=0.9*0.88)
print("Option1 price=",MyPricer.get_price())
MyPricer=FXPricer(r_d=1.24/100, r_f=2.29/100, sigm=0.06, T=365/365, S0=0.88,K=1.1*0.88)
print("Option2 price=",MyPricer.get_price())

Option1 price= 0.07903280140909552
Option2 price= 0.0008506918141392636


In [11]:
MyPricer=FXPricer(r_d=1.24/100, r_f=2.29/100, sigm=0.06, T=365/365, S0=0.88,K=0.9*0.88)
print("Option1 delta=",MyPricer.get_delta())
print("Option1 gamma=",MyPricer.get_gamma())
print("Option1 theta=",MyPricer.get_gamma())
MyPricer=FXPricer(r_d=1.24/100, r_f=2.29/100, sigm=0.06, T=365/365, S0=0.88,K=1.1*0.88)
print("Option2 delta=",MyPricer.get_delta())
print("Option2 gamma=",MyPricer.get_gamma())
print("Option2 theta=",MyPricer.get_theta())

Option1 delta= 0.9249845320146222
Option1 gamma= 2.017241954959188
Option1 theta= 2.017241954959188
Option2 delta= 0.0405635302049679
Option2 gamma= 1.6436004555846502
Option2 theta= -0.0023441178718983462
